In [1]:
import pandas as pd
import random
import os
os.environ["WANDB_DISABLED"] = "true"

# Seed for reproducibility
random.seed(42)

# Templates for text generation
attack_templates = [
    "Suspicious connection with protocol TCP, service SSH and {bytes} bytes transferred.",
    "Detected anomaly: protocol UDP, flag S0, duration {duration}s.",
    "Attack signature matched on port 21 with {bytes} bytes sent.",
    "Multiple failed login attempts from source using {service} service.",
    "High volume traffic with {bytes} bytes; likely port scan activity."
]

normal_templates = [
    "Normal web browsing session over HTTP with {bytes} bytes.",
    "Routine DNS query using UDP protocol; duration {duration}s.",
    "Successful login over SSH service with {bytes} bytes transferred.",
    "Standard email traffic detected using SMTP.",
    "Low-volume network activity recorded; no anomalies found."
]

def generate_sample(label):
    if label == 1:
        template = random.choice(attack_templates)
    else:
        template = random.choice(normal_templates)

    return {
        "text": template.format(
            bytes=random.randint(100, 10000),
            duration=random.randint(1, 60),
            service=random.choice(['FTP', 'SSH', 'Telnet', 'SMTP', 'HTTP'])
        ),
        "label": label
    }

# Generate dataset
data = [generate_sample(label) for label in ([1]*50 + [0]*50)]
random.shuffle(data)  # Shuffle to mix labels

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
#df.to_csv("synthetic_network_dataset.csv", index=False)

# Show a few samples
print(df.head())


                                                text  label
0  High volume traffic with 4404 bytes; likely po...      1
1  High volume traffic with 4471 bytes; likely po...      1
2  Routine DNS query using UDP protocol; duration...      0
3  Routine DNS query using UDP protocol; duration...      0
4  Attack signature matched on port 21 with 206 b...      1


In [2]:
df.head()

,text,label
0,High volume traffic with 4404 bytes; likely po...,1
1,High volume traffic with 4471 bytes; likely po...,1
2,Routine DNS query using UDP protocol; duration...,0
3,Routine DNS query using UDP protocol; duration...,0
4,Attack signature matched on port 21 with 206 b...,1


In [3]:
df.shape

(100, 2)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
df['label'] = df['label'].astype(int)

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare HuggingFace Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function)
test_dataset = test_dataset.map(tokenize_function)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=10, training_loss=0.6647886276245117, metrics={'train_runtime': 15.4823, 'train_samples_per_second': 5.167, 'train_steps_per_second': 0.646, 'total_flos': 10597391892480.0, 'train_loss': 0.6647886276245117, 'epoch': 1.0})

In [7]:
sample_texts = df['text'][:5].tolist()
print(type(sample_texts))          # Should be <class 'list'>
print(all(isinstance(t, str) for t in sample_texts))  # Should be True
print(sample_texts[:3])            # Look at first few texts


<class 'list'>
True
['High volume traffic with 4404 bytes; likely port scan activity.', 'High volume traffic with 4471 bytes; likely port scan activity.', 'Routine DNS query using UDP protocol; duration 52s.']


In [11]:
import shap
from transformers import pipeline

# Create a text classification pipeline
model_name = "distilbert-base-uncased"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name, return_all_scores=True)

# Define the explainer
explainer = shap.Explainer(classifier)

# Explain a single sample
shap_values = explainer(["protocol tcp service http connection established"])

# Visualize
shap.plots.text(shap_values)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
from sklearn.metrics import accuracy_score

preds = trainer.predict(test_dataset)
labels = preds.label_ids
predictions = preds.predictions.argmax(-1)

df_results = test_df.copy()
df_results['pred'] = predictions

accuracy_by_label = df_results.groupby('label').apply(lambda x: accuracy_score(x['label'], x['pred']))
print("Accuracy by label:\n", accuracy_by_label)


Accuracy by label:
 label
0    1.0
1    0.3
dtype: float64


/tmp/ipython-input-9-2735876074.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_label = df_results.groupby('label').apply(lambda x: accuracy_score(x['label'], x['pred']))
